In [ ]:
#### Entropy



In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train_flat = y_train.flatten()

# Set random seed for reproducibility
np.random.seed(42)

# Choose 5 random classes to undersample
all_classes = np.arange(10)
low_classes = np.random.choice(all_classes, 5, replace=False)
high_classes = np.setdiff1d(all_classes, low_classes)

# Create imbalanced training set
x_new, y_new = [], []

for cls in all_classes:
    idx = np.where(y_train_flat == cls)[0]
    if cls in low_classes:
        selected_idx = idx[:500]
    else:
        selected_idx = idx
    x_new.append(x_train[selected_idx])
    y_new.append(y_train[selected_idx])

# Combine and shuffle the imbalanced training data
x_train = np.concatenate(x_new, axis=0)
y_train = np.concatenate(y_new, axis=0)

# Final shuffle
final_idx = np.random.permutation(len(x_train))
x_train = x_train[final_idx]
y_train = y_train[final_idx]

# Normalize and one-hot encode
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



def create_classifier(input_shape=(32, 32, 3), num_classes=10):
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_shape=input_shape, pooling="avg")
    x = base_model.output
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    classifier = models.Model(inputs=base_model.input, outputs=predictions)
    classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return classifier

classifier = create_classifier()
#classifier.summary()

# Initial training
num_initial_samples = 40000
classifier.fit(x_train[:num_initial_samples], y_train[:num_initial_samples], epochs=10)

score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


# Active learning
num_iterations = 10
num_queries = 500

x_train_unlabeled = x_train[num_initial_samples:]
y_train_unlabeled = y_train[num_initial_samples:]

x_train = x_train[:num_initial_samples] 
y_train = y_train[:num_initial_samples]

for iteration in range(num_iterations):
    # Compute entropy for each sample
    probabilities = classifier.predict(x_train_unlabeled)
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-9), axis=1)
    
    # Select samples with highest entropy
    query_indices = np.argsort(entropy)[-num_queries:]
    x_query = x_train_unlabeled[query_indices]
    y_query = y_train_unlabeled[query_indices]
    
    # Remove queried samples from the unlabeled pool
    x_train_unlabeled = np.delete(x_train_unlabeled, query_indices, axis=0)
    y_train_unlabeled = np.delete(y_train_unlabeled, query_indices, axis=0)
    
    # Add queried samples to the training dataset
    x_train = np.concatenate([x_train, x_query], axis=0)
    y_train = np.concatenate([y_train, y_query], axis=0)
    
    # Train the classifier on the updated training dataset
    classifier.fit(x_train, y_train, epochs=5)
    
    score = classifier.evaluate(x_test, y_test, verbose=0)
    print("Test iter:", iteration)
    print("Test accuracy:", score[1])


# Evaluate the classifier
score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Epoch 1/10
860/860 [==============================] - 88s 86ms/step - loss: 1.4230 - accuracy: 0.5360
Epoch 2/10
860/860 [==============================] - 72s 84ms/step - loss: 1.0880 - accuracy: 0.6516
Epoch 3/10
860/860 [==============================] - 70s 81ms/step - loss: 1.1323 - accuracy: 0.6382
Epoch 4/10
860/860 [==============================] - 72s 84ms/step - loss: 1.2426 - accuracy: 0.5846
Epoch 5/10
860/860 [==============================] - 75s 87ms/step - loss: 1.1602 - accuracy: 0.6044
Epoch 6/10
860/860 [==============================] - 70s 81ms/step - loss: 1.3422 - accuracy: 0.5297
Epoch 7/10
860/860 [==============================] - 71s 82ms/step - loss: 1.2274 - accuracy: 0.5913
Epoch 8/10
860/860 [==============================] - 68s 79ms/step - loss: 1.1186 - accuracy: 0.6235
Epoch 9/10
860/860 [==============================] - 71s 83ms/step - loss: 1.1576 - accuracy: 0.6108
Epoch 10/10
860/860 [==============================] - 75s 87ms/step - loss: 1.194

ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
31.2, 30.7, 47.9

In [ ]:
#### Confidence

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50V2
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# Load CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train_flat = y_train.flatten()

# Set random seed for reproducibility
np.random.seed(42)

# Choose 5 random classes to undersample
all_classes = np.arange(10)
low_classes = np.random.choice(all_classes, 5, replace=False)
high_classes = np.setdiff1d(all_classes, low_classes)

# Create imbalanced training set
x_new, y_new = [], []

for cls in all_classes:
    idx = np.where(y_train_flat == cls)[0]
    if cls in low_classes:
        selected_idx = idx[:500]
    else:
        selected_idx = idx
    x_new.append(x_train[selected_idx])
    y_new.append(y_train[selected_idx])

# Combine and shuffle the imbalanced training data
x_train = np.concatenate(x_new, axis=0)
y_train = np.concatenate(y_new, axis=0)

# Final shuffle
final_idx = np.random.permutation(len(x_train))
x_train = x_train[final_idx]
y_train = y_train[final_idx]

# Normalize and one-hot encode
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



def create_classifier(input_shape=(32, 32, 3), num_classes=10):
    base_model = ResNet50V2(include_top=False, weights="imagenet", input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
    predictions = layers.Dense(num_classes, activation='softmax')(x)
    classifier = models.Model(inputs=base_model.input, outputs=predictions)
    classifier.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return classifier

classifier = create_classifier()
#classifier.summary()

# Initial training
num_initial_samples = 40000
classifier.fit(x_train[:num_initial_samples], y_train[:num_initial_samples], epochs=10)

score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

# Active learning
num_iterations = 10
num_queries = 500

x_train_unlabeled = x_train[num_initial_samples:]
y_train_unlabeled = y_train[num_initial_samples:]

x_train = x_train[:num_initial_samples] 
y_train = y_train[:num_initial_samples]

for iteration in range(num_iterations):
    # Compute confidence for each sample
    probabilities = classifier.predict(x_train_unlabeled)
    confidence = np.max(probabilities, axis=1)

    # Select samples with lowest confidence
    query_indices = np.argsort(confidence)[:num_queries]
    x_query = x_train_unlabeled[query_indices]
    y_query = y_train_unlabeled[query_indices]

    # Remove queried samples from the unlabeled pool
    x_train_unlabeled = np.delete(x_train_unlabeled, query_indices, axis=0)
    y_train_unlabeled = np.delete(y_train_unlabeled, query_indices, axis=0)

    # Add queried samples to the training dataset
    x_train = np.concatenate([x_train, x_query], axis=0)
    y_train = np.concatenate([y_train, y_query], axis=0)

    # Train the classifier on the updated training dataset
    classifier.fit(x_train, y_train, epochs=5)
    
    score = classifier.evaluate(x_test, y_test, verbose=0)
    print("Test iter:", iteration)
    print("Test accuracy:", score[1])

# Evaluate the classifier
score = classifier.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])


In [ ]:
22.1, 16.4, 41.3